# Diamond Company Analytics Challenge

**Time:** 35 minutes

**Scenario:** You are the new data analyst at a prestigious diamond company. The CEO has called an urgent meeting and needs answers to critical business questions about the inventory.

**Your Mission:** Analyze the diamond inventory data and provide insights to guide strategic decisions.

In [2]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


## Section 1: Initial Inventory Assessment (5 minutes)

The CEO walks in: "Before we dive into specifics, I need to understand what we're working with. Give me the basics about our inventory database."

### Question 1: "How extensive is our inventory database?"

The CEO needs to know the scale of data you're analyzing.

In [12]:
# Your code here:

import polars as pl
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print(df.shape)

(6000, 8)


### Question 2: "What information do we track about each diamond?"

List all the characteristics we record.

In [5]:
# Your code here:

print(df.columns)

['Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price']


### Question 3: "When I look at one line in this database, what am I looking at?"

Explain in business terms:

**Your answer:** Each row represents:

> Each row represents the detailed information of a specific diamond in the company's inventory, including characeristics like cut, weight, color, etc.



## Section 2: Strategic Business Questions (20 minutes)

The CEO continues: "Now let's get to the important questions. Our board meeting is tomorrow and I need concrete numbers."

### Question 4: "We're considering expanding our storage facilities. How many total carats do we have in inventory? I need to know if our vaults can handle the weight."

Think about what measurement would help answer this.

In [6]:
# Your code here:

df.select(pl.col("Carat Weight").sum().alias("total_carats"))

total_carats
f64
8007.12


### Question 5: "Our new marketing campaign claims we have 'the widest variety of cuts in the industry.' Can you verify this?"

The CEO wants to know about variety, not quantity.

In [13]:
# Your code here:

df.select(pl.col("Cut").n_unique().alias("unique_cuts"))

unique_cuts
u32
5


### Question 6: "I'm meeting with our insurance company. They need to know our price exposure - what's our most and least valuable diamond?"

In [14]:
# Your code here:

df.select([
    pl.col("Price").max().alias("max_price"),
    pl.col("Price").min().alias("min_price")
])

max_price,min_price
i64,i64
101561,2184


### Question 7: "Which color grades should we focus on in our premium line? I need typical prices across our color spectrum."

The CEO wants to understand pricing patterns.

In [18]:
# Your code here:

df.group_by("Color").agg(
    pl.col("Price").mean().alias("avg_price")
).sort("avg_price", descending=True)

Color,avg_price
str,f64
"""D""",15255.783661
"""F""",12712.241856
"""G""",12520.050633
"""E""",11539.190231
"""H""",10487.347544
"""I""",8989.636364


**Reflection:** After analyzing by color, how has your view of the data changed? What does each row in your result represent now?

**Your answer:** New grain =
one color grade. After grouping by color, each row now represents one specific color grade and its average price. This shows the typical price for each color in the inventory.





## Section 3: Advanced Strategic Analysis (10 minutes)

The CEO leans forward: "These next questions will shape our strategy for next year."

### Question 8: "We're thinking of specializing in certain clarity grades. For our investor deck, I need a complete breakdown showing: how many diamonds we have in each clarity grade, their typical price, and the total carats we're holding for each grade."

In [26]:
# Your code here:

df.group_by("Clarity").agg([
    pl.count().alias("diamond_count"),
    pl.col("Price").mean().alias("avg_price"),
    pl.col("Carat Weight").sum().alias("total_carats")
]).sort("avg_price", descending=True)

/tmp/ipython-input-2995871340.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("diamond_count"),


Clarity,diamond_count,avg_price,total_carats
str,u32,f64,f64
"""FL""",4,63776.0,7.87
"""IF""",219,22105.844749,316.0
"""VVS1""",285,16845.680702,389.43
"""VVS2""",666,14142.177177,876.58
"""VS1""",1192,13694.113255,1682.74
"""VS2""",1575,11809.053333,2170.81
"""SI1""",2059,8018.864012,2563.69


### Question 9: "My luxury competitor claims they have the best prices for every cut-color combination. Show me our pricing structure across these combinations - I want to see where we're competitive."

In [27]:
# Your code here:

df.group_by(["Cut", "Color"]).agg(
    pl.col("Price").mean().alias("avg_price")
).sort(["Cut", "avg_price"], descending=[False, True])

Cut,Color,avg_price
str,str,f64
"""Fair""","""G""",7345.52381
"""Fair""","""F""",6063.625
"""Fair""","""D""",6058.25
"""Fair""","""H""",5908.5
"""Fair""","""E""",5370.625
…,…,…
"""Very Good""","""F""",12413.905495
"""Very Good""","""G""",12354.013841
"""Very Good""","""E""",12101.910217


**Reflection:** How specific is your analysis now? What level of detail does each row represent?

**Your answer:** New grain =
one cut&color combination. After grouping by both cut and color, each row in the result now represents a unique pairing of cut and color, along with the pricing information for that combination.

### Question 10: URGENT - "Our biggest client is interested in purchasing entire inventory segments. They asked: 'Which color-cut combinations represent your highest total inventory value?' Show me the top 5 most valuable inventory segments by total worth, and include how many diamonds are in each segment."

This is critical for tomorrow's negotiation.

In [3]:
# Your code here:

df.group_by(["Color", "Cut"]).agg([
    pl.col("Price").sum().alias("total_value"),
    pl.count().alias("diamond_count")
]).sort("total_value", descending=True).head(5)

/tmp/ipython-input-2189829545.py:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("diamond_count")


Color,Cut,total_value,diamond_count
str,str,i64,u32
"""G""","""Ideal""",9363514,690
"""G""","""Very Good""",7140620,578
"""F""","""Very Good""",5648327,455
"""F""","""Ideal""",5346782,363
"""H""","""Ideal""",5279687,458


## Debrief

Congratulations! You've just completed your first day as the diamond company's data analyst.

Remember:
- CEOs speak in business language, not technical terms
- Always understand what each row represents before analyzing
- Business questions contain clues about which operations to use